Notebook based on "segmentation_example_keras" and "solution" notebooks
the EMBL Machine Learning for Image Analysis Course 2018:

Organisers 
Anna Kreshuk - EMBL, Vera Matser - EMBL EBI, Tobias Rasse - EMBL

Trainers
Thorsten Falk - Freiburg University
Fred Hamprecht - Heidelberg University
Anna Kreshuk - EMBL
Constantin Pape - Heidelberg University
Tobias Rasse - EMBL
Pejman Rasti - Université d’Angers
David Rousseau - Université d’Angers
Martin Weigert - MPI-CBG
Uwe Schmidt - MPI-CBG

modified by Tobias Rasse tobias.rasse@mpi-bn.mpg.de for use in OpSeF

### Importing the needed libraries

In [1]:
import os
import random
import pandas as pd
from itertools import chain
import numpy as np
# h5py to read the data-set
import h5py
# matplotlob for plotting
import matplotlib.pyplot as plt
# tensorboard
from keras.callbacks import TensorBoard

import tensorflow as tf
import keras

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.
/home/trasse/anaconda3/envs/opsef004/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/trasse/anaconda3/envs/opsef004/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/trasse/anaconda3/envs/opsef004/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/trasse/a

In [2]:
import tifffile as tif

In [3]:
import inspect
import pickle
from glob import glob
import os
from csbdeep.utils import Path, normalize
from stardist import fill_label_holes, random_label_cmap, calculate_extents, gputools_available,_draw_polygons
from stardist.models import Config2D, StarDist2D, StarDistData2D
from tqdm import tqdm

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import skimage.segmentation
import skimage.morphology
import skimage.transform
from scipy.ndimage.morphology import binary_dilation

In [5]:
from keras.utils import to_categorical

In [6]:
from skimage.morphology import disk
from skimage.filters import rank

## Functions Pre & Processing

In [7]:
def label_to_target(label, tMS):
    #converts label in background, cell and boundary
    size_filter = tMS["min_cell_size_label"]
    boundary_width = tMS["boundary_width"]
    # next, label the annotations and remove small objects
    label = skimage.morphology.label(label)
    label = skimage.morphology.remove_small_objects(label, min_size=size_filter)
    # next, find boundaries and dilate them
    boundaries = skimage.segmentation.find_boundaries(label)
    boundaries = binary_dilation(boundaries, iterations=boundary_width)
    # we return labels with a single channel, where the values stand for:
    # 0: background
    # 1: inner cells
    # 2: cell boundaries
    label_out = np.zeros_like(label, dtype='int64')
    label_out[(label != 0) & (boundaries == 0)] = 1
    label_out[boundaries == 1] = 2
    return label_out

In [8]:
def load_and_preprocess_data(trainModelSettings,which):
    '''Loads and pre-processes the data
        a list of images is returned'''
    # get filenames
    XN = sorted(glob('{}/{}/images/*.tif'.format(trainModelSettings["path"],which)))
    YN = sorted(glob('{}/{}/masks/*.tif'.format(trainModelSettings["path"],which)))
    assert all(Path(x).name==Path(y).name for x,y in zip(XN,YN))
    
    # load data
    X = list(map(tif.imread,XN))
    Y = list(map(tif.imread,YN))
    n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]
    
    # Normalize images and fill small label holes
    axis_norm = (0,1)   # normalize channels independently
    # axis_norm = (0,1,2) # normalize channels jointly
    if n_channel > 1:
        print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))
        sys.stdout.flush()
    X = [normalize(x,1,99.8,axis=axis_norm) for x in tqdm(X)]
    Y = [fill_label_holes(y) for y in tqdm(Y)]
    return XN,X,Y

In [9]:
def perObjectToBinary(Y,tMS,ch):
    ''' converts lavel from one number per object to the core of objects as binary masks'''
    Y3 = [label_to_target(ll,tMS) for ll in Y]
    Y3binary = [to_categorical(img) for img in Y3]
    if ch == 99:
        return Y3binary
    else:
        Y3 = [img[:,:,ch] for img in Y3binary]
        Y3 = [img[:,:,None] for img in Y3]
        return Y3

In [10]:
def prediction_to_label(pred,ppp,return_cells = False):
    ''' does watershed segmentation based on cell predictions
        if return_cells = True also the thresholded predictions are returned
    '''
    # define generator
    def load_img(pred):
        for i in range(len(pred)):
            yield pred[0][:,:,0]
    prob = load_img(pred)
    seg_masks = []
    cell_masks = []
    for p in prob:
        # probability to binary
        cells = p > ppp["prediction_threshold"]
        if return_cells:
            cell_masks.append(cells)
        # create label
        label_cells = skimage.morphology.label(cells)
        # create distance map & smooth
        distance = distance_transform_edt(cells)
        selem = disk(ppp["smooth_distance"])
        distance_smooth = skimage.filters.median(distance, selem=selem)
        # get local maxima
        local_maxima = skimage.feature.peak_local_max(distance_smooth, indices=False, 
                                                      min_distance = ppp["min_distance"],
                                                      labels = label_cells)
        seeds = skimage.morphology.label(local_maxima)
        labels = skimage.morphology.watershed(-distance_smooth, seeds, mask=cells)
        seg_masks.append(labels)
    if return_cells:
        return seg_masks, cell_masks
    else:
        return seg_masks
    

In [11]:
def split_train_val(XN,X,Y):
    # splits data in training and validation dataset
    # get core part of name (that defines orginal image BEFORE augmentation)
    get_names = list(set([os.path.split(fn)[1].split("_")[0] for fn in XN]))
    assert len(X) > 1, "not enough training data"
    # create random subset
    rng = np.random.RandomState(42)
    ind = rng.permutation(len(get_names))
    n_val = max(1, int(round(0.15 * len(ind))))
    ind_train, ind_val = ind[:-n_val], ind[-n_val:]
    val_names, trn_names = [get_names[i] for i in ind_val],[get_names[i] for i in ind_train]
    # map subset back on orginal images
    val_img_index = [i for i in range(0,len(XN)) if os.path.split(XN[i])[1].split("_")[0] in val_names]
    trn_img_index = [i for i in range(0,len(XN)) if os.path.split(XN[i])[1].split("_")[0] in trn_names]
    X_val, Y_val = [X[i] for i in val_img_index]  , [Y3[i] for i in val_img_index]
    X_trn, Y_trn = [X[i] for i in trn_img_index], [Y3[i] for i in trn_img_index]
    # print results
    print('number of images: %3d' % len(X))
    print('- training:       %3d' % len(X_trn))
    print('- validation:     %3d' % len(X_val))
    return X_val, Y_val,X_trn, Y_trn

In [12]:
def add_rest(tms):
    # add more settings to tms
    tms["root"] = data_main_GT
    tms["path"] = os.path.join(tms["root"],tms["data"])
    if tms["model_type"] == "Pretrained":
        tms["model_load"] = "./models/{}.h5".format(tms["Name_Pretrained"])
        tms["model_path"] = "./models/{}_{}.h5".format(tms["Name_Pretrained"],tms["model_name"])
    else:
        tms["model_path"] = "./models/{}_{}.h5".format(tms["model_type"],tms["model_name"])
    return tms

## Functions U-Net

In [13]:
#Each block of u-net architecture consist of two Convolution layers
# These two layers are written in a function to make our code clean
def conv2d_block(input_tensor, n_filters, av, kernel_size=3):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size),
               padding="same")(input_tensor)
    x = keras.layers.Activation(av)(x)
    # second layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), 
               padding="same")(x)
    x = keras.layers.Activation(av)(x)
    return x

In [14]:
def dice_coefficient(y_true, y_pred):
    eps = 1e-6
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection) / (K.sum(y_true_f * y_true_f) + K.sum(y_pred_f * y_pred_f) + eps)

In [15]:
# The u-net architecture consists of contracting and expansive paths which
# shrink and expands the inout image respectivly. 
# Output image have the same size of input image
def get_unet_basic(input_img, trainModelSettings):
    # extract variables
    activation_out = trainModelSettings["unet_activation_out"]
    activation_conv = trainModelSettings["unet_activation_conv"]
    n_filters = trainModelSettings["n_filters"]
    # contracting path
    c1 = conv2d_block(input_img, n_filters*4, activation_conv, kernel_size=3) #The first block of U-net
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = conv2d_block(p1, n_filters*8,activation_conv, kernel_size=3)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = conv2d_block(p2, n_filters*16,activation_conv, kernel_size=3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = conv2d_block(p3, n_filters*32, activation_conv, kernel_size=3)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    
    c5 = conv2d_block(p4, n_filters*64, activation_conv, kernel_size=3)
    
    # expansive path
    u6 = Conv2DTranspose(n_filters*32, (3, 3), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = conv2d_block(u6, n_filters*32, activation_conv, kernel_size=3)

    u7 = Conv2DTranspose(n_filters*16, (3, 3), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = conv2d_block(u7, n_filters*16, activation_conv, kernel_size=3)

    u8 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = conv2d_block(u8, n_filters*8, activation_conv, kernel_size=3)

    u9 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = conv2d_block(u9, n_filters*4, activation_conv, kernel_size=3)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

In [16]:
# The u-net architecture consists of contracting and expansive paths which
# shrink and expands the inout image respectivly. 
# Output image have the same size of input image
def get_unet_Deep1(input_img, trainModelSettings):
    # extract variables
    activation_out = trainModelSettings["unet_activation_out"]
    activation_conv = trainModelSettings["unet_activation_conv"]
    n_filters = trainModelSettings["n_filters"]
    # contracting path
    c1 = conv2d_block(input_img, n_filters*4, activation_conv, kernel_size=3) #The first block of U-net
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = conv2d_block(p1, n_filters*8,activation_conv, kernel_size=3)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = conv2d_block(p2, n_filters*16,activation_conv, kernel_size=3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = conv2d_block(p3, n_filters*32, activation_conv, kernel_size=3)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    
    c5 = conv2d_block(p4, n_filters*64, activation_conv, kernel_size=3)
    p5 = MaxPooling2D(pool_size=(2, 2)) (c5)
    
    c6 = conv2d_block(p5, n_filters*128, activation_conv, kernel_size=3)
    
    # expansive path
    
    u7 = Conv2DTranspose(n_filters*64, (3, 3), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c5])
    c7 = conv2d_block(u7, n_filters*64, activation_conv, kernel_size=3)

    u8 = Conv2DTranspose(n_filters*32, (3, 3), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c4])
    c8 = conv2d_block(u8, n_filters*32, activation_conv, kernel_size=3)

    u9 = Conv2DTranspose(n_filters*16, (3, 3), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c3])
    c9 = conv2d_block(u9, n_filters*16, activation_conv, kernel_size=3)
    
    u10 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c9)
    u10 = concatenate([u10, c2], axis=3)
    c10 = conv2d_block(u10, n_filters*8, activation_conv, kernel_size=3)

    u11 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c10)
    u11 = concatenate([u11, c1], axis=3)
    c11 = conv2d_block(u11, n_filters*4, activation_conv, kernel_size=3)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c11)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

In [17]:
# The u-net architecture consists of contracting and expansive paths which
# shrink and expands the inout image respectivly. 
# Output image have the same size of input image
def get_unet_Deep1_Drop(input_img, trainModelSettings):
    # extract variables
    activation_out = trainModelSettings["unet_activation_out"]
    activation_conv = trainModelSettings["unet_activation_conv"]
    n_filters = trainModelSettings["n_filters"]
    dr = trainModelSettings["drop"]
    
    # contracting path
    c1 = conv2d_block(input_img, n_filters*4, activation_conv, kernel_size=3) #The first block of U-net
    p1 = MaxPooling2D((2, 2)) (c1)
    p1 = Dropout(dr)(p1)

    c2 = conv2d_block(p1, n_filters*8,activation_conv, kernel_size=3)
    p2 = MaxPooling2D((2, 2)) (c2)
    p2 = Dropout(dr)(p2)

    c3 = conv2d_block(p2, n_filters*16,activation_conv, kernel_size=3)
    p3 = MaxPooling2D((2, 2)) (c3)
    p3 = Dropout(dr)(p3)

    c4 = conv2d_block(p3, n_filters*32, activation_conv, kernel_size=3)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    p4 = Dropout(dr)(p4)
    
    c5 = conv2d_block(p4, n_filters*64, activation_conv, kernel_size=3)
    p5 = MaxPooling2D(pool_size=(2, 2)) (c5)
    p5 = Dropout(dr)(p5)
    
    # Middel
    
    c6 = conv2d_block(p5, n_filters*128, activation_conv, kernel_size=3)
    
    # expansive path
    
    u7 = Conv2DTranspose(n_filters*64, (3, 3), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c5])
    u7 = Dropout(dr)(u7)
    c7 = conv2d_block(u7, n_filters*64, activation_conv, kernel_size=3)

    u8 = Conv2DTranspose(n_filters*32, (3, 3), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c4])
    u8 = Dropout(dr)(u8)
    c8 = conv2d_block(u8, n_filters*32, activation_conv, kernel_size=3)

    u9 = Conv2DTranspose(n_filters*16, (3, 3), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c3])
    u9 = Dropout(dr)(u9)
    c9 = conv2d_block(u9, n_filters*16, activation_conv, kernel_size=3)
    
    u10 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c9)
    u10 = concatenate([u10, c2], axis=3)
    u10 = Dropout(dr)(u10)
    c10 = conv2d_block(u10, n_filters*8, activation_conv, kernel_size=3)

    u11 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c10)
    u11 = concatenate([u11, c1], axis=3)
    u11 = Dropout(dr)(u11)
    c11 = conv2d_block(u11, n_filters*4, activation_conv, kernel_size=3)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c11)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

## Main Code

In [18]:
main_folder = os.path.dirname(os.path.abspath(inspect.stack()[0][1])).replace("TrainUNet","Augment")
file_path = "{}/my_runs/augment_settings_xl.pkl".format(main_folder)
infile = open(file_path,'rb')
parameter = pickle.load(infile)
print("Loading processing pipeline from",file_path)
infile.close()
aug_sets,pre_defined_pipelines,data_main_GT,Datasets_Download = parameter

Loading processing pipeline from /home/trasse/github/OpSeF-IV/Train/Augment/my_runs/augment_settings_xl.pkl


In [19]:
jobs_list = []
jobs_total = 3

In [20]:
# define training settings
if jobs_total > 0:
    trainModelSettings = {}
    trainModelSettings["data"] = "DSB2018_FL_Nuc_Subset_Basic_Nuc_512All" # Dataset to Train on
    trainModelSettings["model_type"] = "UNetDeep1_Drop" # one down- and one up-block more than "UNetBasic"
    trainModelSettings["model_name"] = "Nuc_Border" # give the model a name
    trainModelSettings["epochs"] = 5 # Epochs to be trained
    trainModelSettings["unet_activation_out"]= "sigmoid" # Activation U-Net Output
    trainModelSettings["unet_activation_conv"]= "relu" # Activation Conv Layers
    trainModelSettings["n_filters"] = 4 # number of filter
    trainModelSettings["label_convert"] = "perObjectToCore"
    trainModelSettings["drop"] = 0.2 # define dropout
    trainModelSettings["Name_Pretrained"] = ""
    trainModelSettings["loss"] = "binary_crossentropy"
    trainModelSettings["min_cell_size_label"] = 25
    trainModelSettings["boundary_width"] = 2

    # define post-processing
    post_processing_prm = {}
    post_processing_prm["prediction_threshold"] = 0.5 # defines probability threshold 
    post_processing_prm["smooth_distance"] = 3 # Selem Radius used to smooth distance map
    post_processing_prm["min_distance"] = 20 # minimal distance of local maxima

    jobs_list.append([add_rest(trainModelSettings),post_processing_prm])

In [21]:
# define training settings
if jobs_total > 1:
    trainModelSettings = {}
    trainModelSettings["data"] = "DSB2018_FL_Nuc_Subset_Basic_Nuc_512All" # Dataset to Train on
    trainModelSettings["model_type"] = "UNetDeep1" # one down- and one up-block more than "UNetBasic"
    trainModelSettings["model_name"] = "Nuc_Border" # give the model a name
    trainModelSettings["epochs"] = 5 # Epochs to be trained
    trainModelSettings["unet_activation_out"]= "sigmoid" # Activation U-Net Output
    trainModelSettings["unet_activation_conv"]= "relu" # Activation Conv Layers
    trainModelSettings["n_filters"] = 4 # number of filter
    trainModelSettings["label_convert"] = "perObjectToCore"
    trainModelSettings["drop"] = 0.2 # define dropout
    trainModelSettings["Name_Pretrained"] = ""
    trainModelSettings["loss"] = "binary_crossentropy"
    trainModelSettings["min_cell_size_label"] = 25
    trainModelSettings["boundary_width"] = 2

    # define post-processing
    post_processing_prm = {}
    post_processing_prm["prediction_threshold"] = 0.5 # defines probability threshold 
    post_processing_prm["smooth_distance"] = 3 # Selem Radius used to smooth distance map
    post_processing_prm["min_distance"] = 20 # minimal distance of local maxima

    jobs_list.append([add_rest(trainModelSettings),post_processing_prm])

In [22]:
# define training settings
if jobs_total > 2:
    trainModelSettings = {}
    trainModelSettings["data"] = "DSB2018_FL_Nuc_Subset_Basic_Nuc_512All" # Dataset to Train on
    trainModelSettings["model_type"] = "UNetBasic" # one down- and one up-block more than "UNetBasic"
    trainModelSettings["model_name"] = "Nuc_Border" # give the model a name
    trainModelSettings["epochs"] = 5 # Epochs to be trained
    trainModelSettings["unet_activation_out"]= "sigmoid" # Activation U-Net Output
    trainModelSettings["unet_activation_conv"]= "relu" # Activation Conv Layers
    trainModelSettings["n_filters"] = 4 # number of filter
    trainModelSettings["label_convert"] = "perObjectToCore"
    trainModelSettings["drop"] = 0.2 # define dropout
    trainModelSettings["Name_Pretrained"] = ""
    trainModelSettings["loss"] = "binary_crossentropy"
    trainModelSettings["min_cell_size_label"] = 25
    trainModelSettings["boundary_width"] = 2

    # define post-processing
    post_processing_prm = {}
    post_processing_prm["prediction_threshold"] = 0.5 # defines probability threshold 
    post_processing_prm["smooth_distance"] = 3 # Selem Radius used to smooth distance map
    post_processing_prm["min_distance"] = 20 # minimal distance of local maxima

    jobs_list.append([add_rest(trainModelSettings),post_processing_prm])

In [23]:
trained_models = []

In [24]:
for trainModelSettings,post_processing_prm in jobs_list:
    print("Running",trainModelSettings["model_path"])
    # Load Data
    XN,X,Y = load_and_preprocess_data(trainModelSettings,"train")
    # convert label
    if trainModelSettings["label_convert"] == "perObjectToCore":
        Y3 = perObjectToBinary(Y,trainModelSettings,1) # return core
    if trainModelSettings["label_convert"] == "perObjectToBorder":
        Y3 = perObjectToBinary(Y,trainModelSettings,2) # return core
    if trainModelSettings["label_convert"] == "perObjectToMulti":
        Y3 = perObjectToBinary(Y,trainModelSettings,99) # return core
    # split in train & validate
    X_val, Y_val,X_trn, Y_trn = split_train_val(XN,X,Y)
    # reshape
    # train
    X_trn_arr = np.stack([img[:,:,None] for img in X_trn])
    print(X_trn_arr.shape)
    Y_trn_arr = np.stack([img for img in Y_trn])
    print(Y_trn_arr.shape)
    # validate
    X_val_arr = np.stack([img[:,:,None] for img in X_val])
    print(X_val_arr.shape)
    Y_val_arr = np.stack([img for img in Y_val])
    print(Y_val_arr.shape)
    # Define Input
    input_img = Input((X_trn_arr.shape[1], X_trn_arr.shape[2], 1), name='img')
    # Define or Load Model
    if trainModelSettings["model_type"] == "Pretrained":
        pass # load here model
    else:
        if trainModelSettings["model_type"] == "UNetBasic":
            model = get_unet_basic(input_img,trainModelSettings)
        if trainModelSettings["model_type"] == "UNetDeep1":
            model = get_unet_Deep1(input_img,trainModelSettings)
        if trainModelSettings["model_type"] == "UNetDeep1_Drop":
            model = get_unet_Deep1_Drop(input_img,trainModelSettings)
        model.compile(optimizer=Adam(), loss=trainModelSettings["loss"], metrics=[dice_coefficient])
        model.summary()
    # init tensorboard
    NAME= trainModelSettings["model_type"]
    tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
    # train the model
    results = model.fit(X_trn_arr, Y_trn_arr, batch_size=1, epochs=trainModelSettings["epochs"], 
                        callbacks=[tensorboard], validation_data=(X_val_arr, Y_val_arr))

    # Save model
    model.save(trainModelSettings["model_path"])
    print("Saving model to ",trainModelSettings["model_path"])
    trained_models.append(trainModelSettings["model_path"])
    

    
    

Running ./models/UNetDeep1_Drop_Nuc_Border.h5


100%|██████████| 693/693 [00:05<00:00, 122.29it/s]
/home/trasse/anaconda3/envs/opsef004/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  import sys


number of images: 693
- training:       585
- validation:     108
(585, 512, 512, 1)
(585, 512, 512, 1)
(108, 512, 512, 1)
(108, 512, 512, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 16) 160         img[0][0]                        
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 512, 512, 16) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 512, 512, 16) 2320        a

Train on 585 samples, validate on 108 samples
Epoch 1/5
585/585 [==============================] - 28s 47ms/step - loss: 0.1357 - dice_coefficient: 0.8319 - val_loss: 0.0755 - val_dice_coefficient: 0.9055
Epoch 2/5
585/585 [==============================] - 24s 41ms/step - loss: 0.0913 - dice_coefficient: 0.8938 - val_loss: 0.0542 - val_dice_coefficient: 0.9299
Epoch 3/5
585/585 [==============================] - 24s 41ms/step - loss: 0.0912 - dice_coefficient: 0.8948 - val_loss: 0.0687 - val_dice_coefficient: 0.9262
Epoch 4/5
585/585 [==============================] - 24s 41ms/step - loss: 0.0742 - dice_coefficient: 0.9125 - val_loss: 0.0771 - val_dice_coefficient: 0.9218
Epoch 5/5
585/585 [==============================] - 24s 41ms/step - loss: 0.0699 - dice_coefficient: 0.9199 - val_loss: 0.0462 - val_dice_coefficient: 0.9426
Saving model to  ./models/UNetDeep1_Drop_Nuc_Border.h5
Running ./models/UNetDeep1_Nuc_Border.h5


100%|██████████| 693/693 [00:05<00:00, 120.12it/s]
/home/trasse/anaconda3/envs/opsef004/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  import sys


number of images: 693
- training:       585
- validation:     108
(585, 512, 512, 1)
(585, 512, 512, 1)
(108, 512, 512, 1)
(108, 512, 512, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 512, 512, 16) 160         img[0][0]                        
__________________________________________________________________________________________________
activation_23 (Activation)      (None, 512, 512, 16) 0           conv2d_24[0][0]                  
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 512, 512, 16) 2320        a

Train on 585 samples, validate on 108 samples
Epoch 1/5
585/585 [==============================] - 22s 38ms/step - loss: 0.1358 - dice_coefficient: 0.8300 - val_loss: 0.0705 - val_dice_coefficient: 0.9092
Epoch 2/5
585/585 [==============================] - 21s 37ms/step - loss: 0.0843 - dice_coefficient: 0.8993 - val_loss: 0.0657 - val_dice_coefficient: 0.9248
Epoch 3/5
585/585 [==============================] - 21s 37ms/step - loss: 0.0769 - dice_coefficient: 0.9100 - val_loss: 0.0534 - val_dice_coefficient: 0.9335
Epoch 4/5
585/585 [==============================] - 21s 37ms/step - loss: 0.0700 - dice_coefficient: 0.9196 - val_loss: 0.0438 - val_dice_coefficient: 0.9438
Epoch 5/5
585/585 [==============================] - 21s 37ms/step - loss: 0.0906 - dice_coefficient: 0.9216 - val_loss: 0.0437 - val_dice_coefficient: 0.9435
Saving model to  ./models/UNetDeep1_Nuc_Border.h5
Running ./models/UNetBasic_Nuc_Border.h5


100%|██████████| 693/693 [00:05<00:00, 120.29it/s]


number of images: 693
- training:       585
- validation:     108
(585, 512, 512, 1)
(585, 512, 512, 1)
(108, 512, 512, 1)
(108, 512, 512, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
conv2d_47 (Conv2D)              (None, 512, 512, 16) 160         img[0][0]                        
__________________________________________________________________________________________________
activation_45 (Activation)      (None, 512, 512, 16) 0           conv2d_47[0][0]                  
__________________________________________________________________________________________________
conv2d_48 (Conv2D)              (None, 512, 512, 16) 2320        a

Train on 585 samples, validate on 108 samples
Epoch 1/5
585/585 [==============================] - 18s 30ms/step - loss: 0.1183 - dice_coefficient: 0.8457 - val_loss: 0.0683 - val_dice_coefficient: 0.9204
Epoch 2/5
585/585 [==============================] - 17s 29ms/step - loss: 0.0758 - dice_coefficient: 0.9120 - val_loss: 0.0462 - val_dice_coefficient: 0.9384
Epoch 3/5
585/585 [==============================] - 17s 29ms/step - loss: 0.0604 - dice_coefficient: 0.9313 - val_loss: 0.0410 - val_dice_coefficient: 0.9439
Epoch 4/5
585/585 [==============================] - 17s 29ms/step - loss: 0.0587 - dice_coefficient: 0.9332 - val_loss: 0.0376 - val_dice_coefficient: 0.9500
Epoch 5/5
585/585 [==============================] - 17s 29ms/step - loss: 0.0540 - dice_coefficient: 0.9381 - val_loss: 0.0383 - val_dice_coefficient: 0.9499
Saving model to  ./models/UNetBasic_Nuc_Border.h5


In [25]:
lklkl

NameError: name 'lklkl' is not defined

In [ ]:
### Anaylse and inspect results

In [ ]:
# Load test data
XN_test,X_test,Y_test = load_and_preprocess_data(trainModelSettings,"test")

In [ ]:
X_test_arr = np.stack([img[:,:,None] for img in X_test])
Y_trn_arr = np.stack([img for img in Y_test])

In [ ]:
preds_test = model.predict(np.stack(X_test_arr), verbose=1)

In [ ]:
sm, cm = prediction_to_label(preds_test,post_processing_prm, True)

In [ ]:
i = 0

plt.figure(figsize=(15,15))
plt.subplot(221)  #sublot(Anzahl Zeilen Anzahl Spalten Bild Nummer)\
plt.imshow(X_test[i])
#plt.colorbar()
plt.title('Input')

plt.subplot(222)  #sublot(Anzahl Zeilen Anzahl Spalten Bild Nummer)\
plt.imshow(Y_test[i])
#plt.colorbar()
plt.title('Label')

plt.subplot(223)  #sublot(Anzahl Zeilen Anzahl Spalten Bild Nummer)\
plt.imshow(cm[i])
plt.title('Prediction')

plt.subplot(224)  #sublot(Anzahl Zeilen Anzahl Spalten Bild Nummer)\
plt.imshow(sm[i])
plt.title('Predicted Segmentation')
#plt.colorbar()

plt.show();